In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
df = pd.read_csv('../data/raw/GDSC_DATASET.csv')
df.head()

,COSMIC_ID,CELL_LINE_NAME,TCGA_DESC,DRUG_ID,DRUG_NAME,LN_IC50,AUC,Z_SCORE,GDSC Tissue descriptor 1,GDSC Tissue descriptor 2,Cancer Type (matching TCGA label),Microsatellite instability Status (MSI),Screen Medium,Growth Properties,CNA,Gene Expression,Methylation,TARGET,TARGET_PATHWAY
0,683667,PFSK-1,MB,1003,Camptothecin,-1.463887,0.930220,0.433123,nervous_system,medulloblastoma,MB,MSS/MSI-L,R,Adherent,Y,Y,Y,TOP1,DNA replication
1,684057,ES5,UNCLASSIFIED,1003,Camptothecin,-3.360586,0.791072,-0.599569,bone,ewings_sarcoma,NaN,MSS/MSI-L,R,Adherent,Y,Y,Y,TOP1,DNA replication
2,684059,ES7,UNCLASSIFIED,1003,Camptothecin,-5.044940,0.592660,-1.516647,bone,ewings_sarcoma,NaN,MSS/MSI-L,R,Adherent,Y,Y,Y,TOP1,DNA replication
3,684062,EW-11,UNCLASSIFIED,1003,Camptothecin,-3.741991,0.734047,-0.807232,bone,ewings_sarcoma,NaN,MSS/MSI-L,R,Adherent,Y,Y,Y,TOP1,DNA replication
4,684072,SK-ES-1,UNCLASSIFIED,1003,Camptothecin,-5.142961,0.582439,-1.570016,bone,ewings_sarcoma,NaN,MSS/MSI-L,R,Semi-Adherent,Y,Y,Y,TOP1,DNA replication


In [14]:
# Checking the datatypes of the columns in the dataset
df.dtypes

COSMIC_ID                                    int64
CELL_LINE_NAME                              object
TCGA_DESC                                   object
DRUG_ID                                      int64
DRUG_NAME                                   object
LN_IC50                                    float64
AUC                                        float64
Z_SCORE                                    float64
GDSC Tissue descriptor 1                    object
GDSC Tissue descriptor 2                    object
Microsatellite instability Status (MSI)     object
Screen Medium                               object
Growth Properties                           object
CNA                                         object
Gene Expression                             object
Methylation                                 object
TARGET                                      object
TARGET_PATHWAY                              object
dtype: object

In [4]:
# Checking for duplicates in the dataset
print(f'There are {df.duplicated().sum()} duplicate(s) in the Dataset')

There are 0 duplicate(s) in the Dataset


In [5]:
# Checking for null values
df.isna().sum()

COSMIC_ID                                      0
CELL_LINE_NAME                                 0
TCGA_DESC                                   1067
DRUG_ID                                        0
DRUG_NAME                                      0
LN_IC50                                        0
AUC                                            0
Z_SCORE                                        0
GDSC Tissue descriptor 1                    9366
GDSC Tissue descriptor 2                    9366
Cancer Type (matching TCGA label)          51446
Microsatellite instability Status (MSI)    12353
Screen Medium                               9366
Growth Properties                           9366
CNA                                         9366
Gene Expression                             9366
Methylation                                 9366
TARGET                                     27155
TARGET_PATHWAY                                 0
dtype: int64

In [6]:
# Imputing "UNCLASSIFIED" for any blank values in TCGA_DESC
df['TCGA_DESC'] = df["TCGA_DESC"].fillna("UNCLASSIFIED")

In [9]:
# Dropping the Cancer Type column as it is similar to the TCGA_DESC column
df.drop(columns="Cancer Type (matching TCGA label)", inplace=True)

In [12]:
# Replacing the null values in the remaining columns with "UNKNOWN"
columns_with_blanks = ['GDSC Tissue descriptor 1',
       'GDSC Tissue descriptor 2', 'Microsatellite instability Status (MSI)',
       'Screen Medium', 'Growth Properties', 'CNA', 'Gene Expression',
       'Methylation', 'TARGET']

for column in columns_with_blanks:
    df[column] = df[column].fillna("UNKNOWN")

In [13]:
# Checking for the null values in the dataset again
df.isna().sum()

COSMIC_ID                                  0
CELL_LINE_NAME                             0
TCGA_DESC                                  0
DRUG_ID                                    0
DRUG_NAME                                  0
LN_IC50                                    0
AUC                                        0
Z_SCORE                                    0
GDSC Tissue descriptor 1                   0
GDSC Tissue descriptor 2                   0
Microsatellite instability Status (MSI)    0
Screen Medium                              0
Growth Properties                          0
CNA                                        0
Gene Expression                            0
Methylation                                0
TARGET                                     0
TARGET_PATHWAY                             0
dtype: int64

In [10]:
df.columns

Index(['COSMIC_ID', 'CELL_LINE_NAME', 'TCGA_DESC', 'DRUG_ID', 'DRUG_NAME',
       'LN_IC50', 'AUC', 'Z_SCORE', 'GDSC Tissue descriptor 1',
       'GDSC Tissue descriptor 2', 'Microsatellite instability Status (MSI)',
       'Screen Medium', 'Growth Properties', 'CNA', 'Gene Expression',
       'Methylation', 'TARGET', 'TARGET_PATHWAY'],
      dtype='object')

In [16]:
# Exporting the cleaned Dataset
df.to_csv("cleaned_GDSC.csv")

In [23]:
# Exploring the dataset 
# including only float dtype, as they are the important numerical data
# and there is no use in getting the mean, median, mode of ID's
df.describe(include=['float'])

,LN_IC50,AUC,Z_SCORE
count,242035.000000,242035.000000,242035.000000
mean,2.817111,0.882593,0.000006
std,2.762191,0.146998,0.999390
min,-8.747724,0.006282,-8.254501
25%,1.508054,0.849452,-0.656842
50%,3.236744,0.944197,0.010585
75%,4.700111,0.974934,0.656036
max,13.820189,0.998904,7.978776


In [30]:
# This is the python process to export the dataset from python directly into a SQL Schema 
# The password is changed later.
# The host and port are most likely are going to be the same if the database is in local system and you haven't changed during installation

username = 'root'
password = "test" # If you have '@' in the password, it will raise an error,
                  # replace '@' with '%40', it will work. eg., 'test@1234' -> 'test%401234'
host = 'localhost:3306'
database = 'practice'

# Create the connection URL
connection_url = f'mysql+pymysql://{username}:{password}@{host}/{database}'

try:
    # Create an SQLAlchemy engine
    engine = create_engine(connection_url)
    print("Connected to the database successfully.")

    # Create a DataFrame to upload
    # The dataset should be in the same folder as the ipynb file
    accounts_df = pd.read_csv('cleaned_GDSC.csv')
    print("CSV file loaded successfully.")

    # Upload the DataFrame to MySQL
    accounts_df.to_sql("cleaned_gdsc", con=engine, if_exists='replace', index=False)
    print("Data uploaded successfully.")

except Exception as e:
    print("An error occurred:", e)

Connected to the database successfully.
CSV file loaded successfully.
Data uploaded successfully.
